| SOURCE_TABLE_NAME | SOURCE_COLUMN_NAME | REPORTING_TABLE_NAME | REPORTING_COLUMN_NAME | TRANSFORMATION RULE | CONDITIONS |
| --- | --- |--- | --- |--- |--- |
| silver.daily_pricing_silver	| state_name	|***REPORTING_DIM_STATE_GOLD***	|***STATE_NAME***	| Select Unique state_name Values | Identify New/Changed Records From the Source Table . Use ***lakehouse_updated_date Column*** in source table to Identify New/Changed Records|
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***STATE_ID***	| Generate Running Sequnece ID For Each Unique state_name Values | 1. Make Sure No Duplicate State_Name Values Loaded 2.  When loading new State_Name values in subsquent run , STATE_ID values need to be generated on top of existing MAX(STATE_ID)   |
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***lakehouse_inserted_date***	| Load current_timestamp() | |
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***lakehouse_updated_date***	| Load current_timestamp() | |

In [0]:
USE CATALOG pricing_analytics;

CREATE OR REPLACE TABLE silver.reporting_dim_state_stage_1 AS
SELECT 
  DISTINCT STATE_NAME
FROM  
  silver.daily_pricing_silver
WHERE 
  lake_house_updated_date>(SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS WHERE PROCESS_NAME='reportingDimensionTablesLoad' AND process_status='Completed')

In [0]:
SELECT 
  STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY STATE_NAME) AS STATE_ID,
  current_timestamp(),
  current_timestamp
FROM  
  silver.reporting_dim_state_stage_1 

In [0]:
INSERT INTO gold.reporting_dim_state_gold
SELECT 
  STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY STATE_NAME) AS STATE_ID,
  current_timestamp(),
  current_timestamp
FROM  
  silver.reporting_dim_state_stage_1 

In [0]:
SELECT * FROM GOLD.reporting_dim_state_gold ORDER BY STATE_ID

In [0]:
INSERT INTO gold.reporting_dim_state_gold
SELECT 
  STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY STATE_NAME) AS STATE_ID,
  current_timestamp(),
  current_timestamp
FROM  
  silver.reporting_dim_state_stage_1 

In [0]:
SELECT * FROM GOLD.reporting_dim_state_gold ORDER BY STATE_ID

In [0]:
SELECT 
  silverDim.STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY silverDim.STATE_NAME) AS STATE_ID,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_date
  FROM 
    silver.reporting_dim_state_stage_1 AS silverDim
  LEFT OUTER JOIN
    gold.reporting_dim_state_gold AS goldDim
  ON
    silverDim.STATE_NAME=goldDim.STATE_NAME
  --WHERE goldDim.STATE_NAME IS NULL

In [0]:
SELECT 
  silverDim.STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY silverDim.STATE_NAME) AS STATE_ID,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_date
  FROM 
    silver.reporting_dim_state_stage_1 AS silverDim
  LEFT OUTER JOIN
    gold.reporting_dim_state_gold AS goldDim
  ON
    silverDim.STATE_NAME=goldDim.STATE_NAME
WHERE goldDim.STATE_NAME IS NULL

In [0]:
TRUNCATE TABLE gold.reporting_dim_state_gold

In [0]:
DELETE FROM gold.reporting_dim_state_gold WHERE STATE_ID IN(24,25)

In [0]:
SELECT 
  silverDim.STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY silverDim.STATE_NAME) AS STATE_ID,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_date
  FROM 
    silver.reporting_dim_state_stage_1 AS silverDim
  LEFT OUTER JOIN
    gold.reporting_dim_state_gold AS goldDim
  ON
    silverDim.STATE_NAME=goldDim.STATE_NAME
WHERE goldDim.STATE_NAME IS NULL

In [0]:
CREATE OR REPLACE TABLE silver.reporting_dim_state_stage_2 AS
SELECT 
  silverDim.STATE_NAME,
  ROW_NUMBER() OVER (ORDER BY silverDim.STATE_NAME) AS STATE_ID,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_date
  FROM 
    silver.reporting_dim_state_stage_1 AS silverDim
  LEFT OUTER JOIN
    gold.reporting_dim_state_gold AS goldDim
  ON
    silverDim.STATE_NAME=goldDim.STATE_NAME
WHERE goldDim.STATE_NAME IS NULL

In [0]:
SELECT * FROM silver.reporting_dim_state_stage_2

In [0]:
SELECT 
  silverDim.STATE_NAME,
  silverDim.STATE_ID + PREV_MAX_SK_ID AS STATE_ID
 FROM 
  silver.reporting_dim_state_stage_2 silverDim
 CROSS JOIN (SELECT MAX(STATE_ID) AS PREV_MAX_SK_ID FROM gold.reporting_dim_state_gold) goldDim

In [0]:
SELECT 
  silverDim.STATE_NAME,
  silverDim.STATE_ID + PREV_MAX_SK_ID AS STATE_ID,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_date
 FROM 
  silver.reporting_dim_state_stage_2 silverDim
 CROSS JOIN (SELECT NVL(MAX(STATE_ID),0) AS PREV_MAX_SK_ID FROM gold.reporting_dim_state_gold) goldDim

In [0]:
CREATE OR REPLACE TABLE silver.reporting_dim_state_stage_3 AS
SELECT 
  silverDim.STATE_NAME,
  silverDim.STATE_ID + PREV_MAX_SK_ID AS STATE_ID,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_date
 FROM 
  silver.reporting_dim_state_stage_2 silverDim
 CROSS JOIN (SELECT NVL(MAX(STATE_ID),0) AS PREV_MAX_SK_ID FROM gold.reporting_dim_state_gold) goldDim

In [0]:
TRUNCATE TABLE gold.reporting_dim_state_gold

In [0]:
INSERT INTO gold.reporting_dim_state_gold
SELECT 
  STATE_NAME,
  STATE_ID,
  current_timestamp(),
  current_timestamp()
FROM 
silver.reporting_dim_state_stage_3